In [3]:
import pandas as pd
from datetime import datetime
import re

rawpath = "../raw"

In [23]:
df = pd.read_excel(f"{rawpath}/riesaufdrucke.xlsx", dtype={"papiermacher_idn":"string","papiermuehle_idn":"string", "quelle":"string", "bezugswerke":"string", "schlagwortfolge":"string", "wz_motiv":"string", "technik":"string", "druckfarbe":"string", "ort_idn":"string", "6800 - IDN Provenienz":"string"})
df = df.apply(lambda x: x.str.strip() if (x.dtype == "object") or (x.dtype == "string") else x)
df

,kasten,signatur,entstehungszeit,entstehungszeit_lang,datentraeger,sprache,land,papiermacher_idn,papiermuehle_idn,papiermacher,...,bezugswerke,schlagwortfolge,wz_motiv,technik,druckfarbe,ort_idn,6800 - IDN Provenienz,provenienz,komm_extern,8034
0,Riesaufdrucke A,1971/Bl 105,1789-1819,1789-1819 (Wirkungszeit des Papiermachers),TB-papier-hr,/1zxx,/1XA-DXDE,1051303966,1037995147,Johann Samuel Kutter,...,<NA>,954480481 940605627,1043027912,40257363,41803329,40486346,<NA>,NaN,Beilage: in der Masse gefärbter blau-grauer Karton mit handschriftlicher Notiz,NaN
1,Riesaufdrucke A,1971/Bl 109,1820-1840,um 1820-1840,TB-papier-hv,/1zxx,/1XA-DXDE,<NA>,1035382296,[Nicht ermittelbar],...,Motiv ähnlich zu RA 1973/Bl 19,954480481 963257161,<NA>,40257363,41803329,960259481,<NA>,NaN,Nachdruck,"Notiz auf Rückseite: Riesumschlag-Druck aus der Papiermühle zu Fischlaken bei Werder/Ruhr, vermutlich von ca. 1820/40. Abdruck 1929 auf altem Velinpapier. Der oberste Teil der Druckplatte ist in früherer Zeit abgebrochen."
2,Riesaufdrucke A,1971/Bl 110,NaN,nach 1743 (Wirkungszeit des Papiermachers),TB-papier-hr,/1zxx,/1XA-AT,1052333095,1043470956,Hans Magensack,...,<NA>,972322698,<NA>,40257363,41803329,951591258,<NA>,NaN,Beilage: Riesaufdruck-Kopie,NaN
3,Riesaufdrucke A,1971/Bl 205,1789-1819,1789-1819 (Wirkungszeit des Papiermachers),TB-papier-hr,/1zxx,/1XA-DXDE,1051303966,1037995147,Johann Samuel Kutter,...,"Motiv identisch zu RA 1971/Bl 206, RA 1971/Bl 207, RA 1971/Bl 208, RA 1975/Bl 132",954901096 041658019 945895291,<NA>,40257363,41803329,40486346,<NA>,NaN,handschriftliche Notiz auf Voderseite: Riesumschlag der Papiermühle von Joh. Sam. Kutter in Ravensburg,NaN
4,Riesaufdrucke A,1971/Bl 206,1789-1819,1789-1819 (Wirkungszeit des Papiermachers),TB-papier-hr,/1zxx,/1XA-DXDE,1051303966,1037995147,Johann Samuel Kutter,...,"Motiv identisch zu RA 1971/Bl 205, RA 1971/Bl 207, RA 1971/Bl 208, RA 1975/Bl 132",954901096 041658019 945895291,<NA>,40257363,41803329,40486346,<NA>,NaN,handriftliche Notiz auf Rückseite: ? 1800,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,Riesaufdrucke C,1972/Bl 24,1850-1972,1850-1972,TB-papier,/1zxx,/1XA-NL,<NA>,<NA>,[Nicht ermittelbar],...,<NA>,41796012 963257161,<NA>,40257363,41784820,975469568,<NA>,NaN,NaN,NaN
118,Riesaufdrucke C,1972/Bl 25,1850-1972,1850-1972,TB-papier,/1zxx,/1XA-NL,<NA>,<NA>,[Nicht ermittelbar],...,<NA>,41164636 954480481,<NA>,40257363,41784820,975469568,<NA>,NaN,NaN,NaN
119,Riesaufdrucke C,1972/Bl 26,1850-1972,1850-1972,TB-papier,/1zxx,/1XA-NL,<NA>,<NA>,[Nicht ermittelbar],...,<NA>,41971094,<NA>,40481662,41784820,975469568,<NA>,NaN,NaN,NaN
120,Riesaufdrucke C,1972/Bl 31,1850-1972,1850-1972,TB-papier,/1zxx,/1XA-NL,<NA>,<NA>,[Nicht ermittelbar],...,<NA>,41144368,<NA>,40257363,41784820,975469568,<NA>,NaN,NaN,NaN


In [24]:
def val_idn(idn):
    if len(idn) < 9:
        return (9 - len(idn)) * '0' + idn
    else:
        return idn

In [28]:
def pica_schreiben(row):
    
    pica = """0500 Pa
0501 unbewegtes Bild$bsti
0501 text$btxt
0502 ohne Hilfsmittel zu benutzen$bn
0503 Blatt$bnb
0600 yy
"""
    if year := re.findall(r"\d{4}", str(row.entstehungszeit)):
        if len(year) == 2:
            pica += f"1100 {year[0]}$n[zwischen {year[0]} und {year[1]}]\n1110 {row.entstehungszeit_lang}*{row.entstehungszeit}$a{year[0]}$b{year[1]}$4vzta\n"
        
        elif len(year) == 1:
            pica += f"1100 {year[0]}$n{row.entstehungszeit}\n1110 {row.entstehungszeit_lang}*{row.entstehungszeit}$4vzta\n"
    
    elif year := re.findall(r"\d{2}X{2}", str(row.entstehungszeit)):
        pica += f"1100 {row.entstehungszeit}\n1110 {row.entstehungszeit_lang}*{row.entstehungszeit}$4vzta\n"
    
    pica += f"""1130 {row.datentraeger}
1131 !041454197!
1132 a1-analog;a2-druck;f1-bild;f2-blatt;o-org
1500 /1zxx
1700 {row.land}
"""
    if pd.notna(row.papiermacher_idn):
        pica += f"3000 !{val_idn(row.papiermacher_idn)}!$BPapiermacher$4ppm\n"
    
    if pd.notna(row.papiermuehle_idn):
        pica += f"3100 !{val_idn(row.papiermuehle_idn)}!$BGeistiger Schöpfer$4cre\n"
    
    # Papiermacher bekannt
    if (row.papiermacher != "[Nicht ermittelbar]"):
        pica += f"4000 Riesaufdruck des Papiermachers {row.papiermacher}, Motiv: {row.motiv}"
        
        # Papiermühle bekannt?
        if row.papiermuehle != "[Nicht ermittelbar]":
            pica += f" / {row.papiermuehle}"
        
        pica += "\n"
      
    # Papiermacher unbekannt
    elif (row.papiermacher == "[Nicht ermittelbar]"):
        pica += f"4000 Riesaufdruck eines unbekannten Papiermachers, Motiv: {row.motiv}"
        
        # Papiermühle bekannt?
        if row.papiermuehle != "[Nicht ermittelbar]":
            pica += f" / {row.papiermuehle}"
        
        pica += "\n"

    pica += f"4019 Druckgrafik$Bobja\n"

    if pd.notna(row.text):
        pica += f"4019 {row.text}$Bdevi\n"
    
    pica += f"""4046 {row.ort} : {row.Papiermuehle_4046}
4060 1 ungezähltes Blatt
"""
    blattmasse = [x.replace(" cm", "").replace(",","") for x in row.blattmass.split(" x ")]
    pica += f"4062 {blattmasse[0]} x {blattmasse[1]} mm$b{blattmasse[0]}$h{blattmasse[1]}$4bltt\n"

    bildmasse = [x.replace(" cm", "").replace(",","") for x in row.bildmass.split(" x ")]
    pica += f"4062 {bildmasse[0]} x {bildmasse[1]} mm$b{bildmasse[0]}$h{bildmasse[1]}$4bld\n"

    if pd.notna(row.plattenmass):
        plattenmasse = [x.replace(" cm", "").replace(",","") for x in row.plattenmass.split(" x ")]
        pica += f"4062 {plattenmasse[0]} x {plattenmasse[1]} mm$b{plattenmasse[0]}$h{plattenmasse[1]}$4drck\n"

    pica += "4105 !1257829130!\n"
    
    if pd.notna(row.anmerkungen):
        pica += f"4201 {row.anmerkungen}\n"
    
    if pd.notna(row.komm_1):
        pica += f"4201 {row.komm_1}\n"
    
    if pd.notna(row.komm_2):
        pica += f"4201 {row.komm_2}\n"

    if pd.notna(row.quelle):
        pica += f"4276 !{val_idn(row.quelle)}!$4lita\n4276 !{val_idn(row.quelle)}!$4abba\n"
    
    if pd.notna(row.bezugswerke):
        pica += f"4276 {val_idn(row.bezugswerke)}$4rela\n"
    
    if pd.notna(row.schlagwortfolge):
        counter = 0
        for schlagwort in row.schlagwortfolge.split("\n"):
            pica += f"510{counter} !{val_idn(schlagwort)}!\n"
            counter += 1
    
    if pd.notna(row.wz_motiv):
        for motiv in row.wz_motiv.split("\n"):
            pica += f"5320 !{val_idn(motiv)}!\n"

    pica += "5590 [Objektgattung]\n5590 !042115752!\n"

    pica += f"""5591 [Technik]
5591 !{val_idn(row.technik)}!
"""
    if pd.notna(row.druckfarbe):
        pica += f"5592 [Druck]\n5592 !041507266!\n"

        for farbe in row.druckfarbe.split("\n"):
            pica += f"5592 !{val_idn(farbe)}!\n"

    if pd.notna(row.ort_idn):
        pica += f"5593 [Entstehungsort]\n5593 !{val_idn(row.ort_idn)}!\n"
    
    pica += f"""7001 {now.strftime('%d-%m-%y')} : x
4800 !009033645!
6710 !1032059397!
"""
    if pd.notna(row['6800 - IDN Provenienz']):
        pica += f"6800 Provenienz]\n6800 !{val_idn(row['6800 - IDN Provenienz'])}!\n6800 row.provenienz\n"
    
    pica += f"7100 PS.RA @ m\n7109 !!DBSM/S!! ; PS.RA {row.signatur}\n"
    
    if pd.notna(row.komm_extern):
        pica += f"8034 {row.komm_extern}\n"

    pica += "\t\n"
    return pica

In [84]:
now = datetime.now()
for index, row in df.sample(2).iterrows():
    print(pica_schreiben(row))

0500 Pa
0501 unbewegtes Bild$bsti
0501 text$btxt
0502 ohne Hilfsmittel zu benutzen$bn
0503 Blatt$bnb
0600 yy
1100 1850$n[zwischen 1850 und 1972]
1110 1850-1972*1850-1972$a1850$b1972$4vzta
1130 TB-papier
1131 !041454197!
1132 a1-analog;a2-druck;f1-bild;f2-blatt;o-org
1500 /1zxx
1700 /1XA-NL
4000 Riesaufdruck eines unbekannten Papiermachers, Motiv: zwei auf einem reich verzierten Podest stehende Löwen halten das gekrönte Wappen der Stadt Amsterdam
4019 Druckgrafik$Bobja
4019 Super Fyn Schryff Papier P. Out Koog a. d. Zaan$Bdevi
4046 Koog aan de Zaan : [Nicht ermittelbar]
4060 1 ungezähltes Blatt
4062 470 x 645 mm$b470$h645$4bltt
4062 163 x 167 mm$b163$h167$4bld
4105 !1257829130!
4201 Holland, Koog a. d. Zaan / Honig / Buchhändler P. Out (1830-1911), Papierverkauf seit 1890 (Kartei)
5100 !954480481!
5101 !041658019!
5102 !041144368!
5590 [Objektgattung]
5590 !042115752!
5591 [Technik]
5591 !40257363!
5592 [Druck]
5592 !041507266!
5592 !41784820!
5593 [Entstehungsort]
5593 975469568
7001 2

In [29]:
now = datetime.now()
with open(f"../dat/riesaufdrucke-{now.strftime('%y-%m-%d-%H-%M-%S')}.dat","w") as f:
    with open(f"../dat/riesaufdrucke-recent.dat", 'w') as f2:
        for index, row in df.iterrows():
            f.write(pica_schreiben(row))
            f2.write(pica_schreiben(row))

with open(f"../dat/riesaufdrucke-recent-sample.dat", 'w') as f:
    for index, row in df.sample(5).iterrows():
        f.write(pica_schreiben(row))